<a href="https://colab.research.google.com/github/SurinSeong/FinalPJT/blob/main/oliveyoung_crawling_%EC%B5%9C%EC%A2%85_%EC%88%98%EB%A6%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 라이브러리 로드

In [ ]:
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

## 함수

In [ ]:
## 제품 상세 페이지 가는 함수 ##
def product_change(i, j):
    try:
        print(f'START >> [{(j+1)+(4*i)}] / [48]')

        # url 저장하기
        url = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({8+i}) > li:nth-child({j+1}) > div > a').get_attribute('href')
        urls.append(url)
        # print('url 저장 완료')

        # 제품 상세 페이지 새창 가기
        driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({8+i}) > li:nth-child({j+1}) > div > p.prd_btn_area > button.btn_new_pop.goodsList').click()
        # time.sleep(0)
        driver.switch_to.window(driver.window_handles[1])

        # 해당 제품 정보 크롤링하기
        try:
            info_about_current_product(pictures, brands, names, prices, sale_prices, volumes, skin_types, ingredients, reviews)
        except:
            print('크롤링 실패')

        # 해당 새 탭 닫기
        driver.close()
        print('='*50)
        driver.switch_to.window(driver.window_handles[0])

    except:
        print(f'{(j+1)+(4*i)} 번째 제품이 존재하지 않습니다.')

        # 해당 새 탭 닫기
        # driver.switch_to.window(driver.window_handles[1])
        driver.close()
        print('='*50)
        # driver.switch_to.window(driver.window_handles[0])

In [ ]:
## 데이터 프레임에 넣고 CSV 저장하는 함수 ##
def make_save(n, urls, pictures, brands, names, prices, sale_prices, volumes, skin_types, ingredients, reviews):
    # if len(reviews) != 48:
    #     for k in range(48-len(reviews)):
    #         reviews.append([])
    raw = pd.DataFrame({'name' : names,
                        'brand' : brands,
                        'price' : prices,
                        'sale_price' : sale_prices,
                        'picture' : pictures,
                        'url' : urls,
                        'volume' : volumes,
                        'skin_type' : skin_types,
                        'ingredient' : ingredients,
                        'review' : reviews})
    print('DF 생성 완료')
    raw.to_csv(f'./page{n}_reviews.csv', index=False)
    print('DF 저장 완료')

In [ ]:
## 현재 메인 페이지의 제품 크롤링 함수 ##
def info_about_current_product(pictures, brands, names, prices, sale_prices, volumes, skin_types, ingredients, reviews):
    # 사진 가져오기
    try:
        picture = driver.find_element(By.CSS_SELECTOR, '#prd_thumb_list > li:nth-child(3) > a').get_attribute('data-img')
        pictures.append(picture)
        # print('두 번째 사진 저장 완료')
    except:
        picture = driver.find_element(By.CSS_SELECTOR, '#prd_thumb_list > li.sel > a').get_attribute('data-img')
        pictures.append(picture)
        # print('첫 번째 사진 저장 완료')

    # 브랜드명 가져오기
    try:
        brand = driver.find_element(By.CSS_SELECTOR, 'div.prd_info > p.prd_brand > #moveBrandShop').text
        brands.append(brand)
        # print('브랜드 저장 완료')
    except:
        brands.append(None)
        print('브랜드 저장 실패')

    # 제품명 가져오기
    try:
        name = driver.find_element(By.CSS_SELECTOR, 'div.prd_info > p.prd_name').text
        names.append(name)
        # print('제품명 저장 완료')
    except:
        names.append(None)
        print('제품명 저장 실패')

    # 판매가 가져오기
    try:
        price = driver.find_element(By.CSS_SELECTOR, 'div.prd_info > div.price > span.price-1 > strike').text
        prices.append(price)
        # print('판매가 저장 완료')
    except:
        prices.append(None)
        print('판매가 저장 실패')

    # 최적가 가져오기
    try:
        sale_price = driver.find_element(By.CSS_SELECTOR, 'div.prd_info > div.price > span.price-2 > strong').text
        sale_prices.append(sale_price)
        # print('최적가 저장 완료')
    except:
        sale_prices.append(None)
        print('최적가 저장 실패')

    ## 구매정보 열기 ##
    driver.find_element(By.CSS_SELECTOR, 'ul#tabList > li#buyInfo > a').click()
    time.sleep(1)

    # 용량 가져오기
    try:
        volume = driver.find_element(By.CSS_SELECTOR, '#artcInfo > dl:nth-child(2) > dd').text
        volumes.append(volume)
        # print('용량 저장 완료')
    except:
        volumes.append(None)
        print('용량 저장 실패')

    # 주요 사양 가져오기
    try:
        skin_type = driver.find_element(By.CSS_SELECTOR, '#artcInfo > dl:nth-child(3) > dd').text
        skin_types.append(skin_type)
        # print('사양 저장 완료')
    except:
        skin_types.append(None)
        print('사양 저장 실패')

    # 주요 성분 가져오기
    try:
        ingredient = driver.find_element(By.CSS_SELECTOR, '#artcInfo > dl:nth-child(8) > dd').text
        ingredients.append(ingredient)
        # print('성분 저장 완료')
    except:
        ingredients.append(None)
        print('성분 저장 실패')

    ## 리뷰정보 열기 ##
    driver.find_element(By.CSS_SELECTOR, 'ul#tabList > li#reviewInfo > a').click()
    time.sleep(1)
    # 최신순 선택
    driver.find_element(By.LINK_TEXT, '최신순').click()
    # 체험단 선택 해제
    driver.find_element(By.ID, 'searchType_3').click()

    ## 리뷰 가져오기 ##
    # 리뷰 페이지 개수 가져오기
    review_page = len(driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a'))

    if review_page != 10:
        product_reviews.append(None)
    else:
        # 리뷰 저장할 리스트
        product_reviews = []

        print('리뷰 저장 시작')

        # 해당 제품 당 100개의 댓글 가져오기 (댓글 페이지당 10개의 댓글 존재)
        for p in range(review_page):
            current_page = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > strong').text
            print(f">> [{current_page}] / [{review_page}]")
            for num in range(10):
                # 리뷰 가져오는 함수 사용
                add_review(num, p, product_reviews)
            # 다음 페이지로 넘어가기
            driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({p+2})').click()
            time.sleep(1)

        # 제품의 리뷰들을 전체 리뷰 리스트에 저장
        reviews.append(product_reviews)
    print('리뷰 저장 완료')

In [ ]:
## 리뷰 가져오기 함수
def add_review(num, page_num, product_reviews): # page number를 변수로 지정
    try:
        review = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({num+1}) > div.review_cont > div.txt_inner').text.replace('\n', ' ')
        # print(num+1, '성공')
        product_reviews.append(review)
    except:
        # error = f'{page_num}-{num+1}'
        product_reviews.append('None')
        print(f'{page_num}-{num+1} 번째 리뷰 저장 실패')

## 메인 페이지에서 총 페이지 개수 확인하는 부분

In [ ]:
# 올리브영 - 스킨케어 - 에센스/세럼/앰플 페이지
# main_url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010016&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat100000100010014_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EC%97%90%EC%84%BC%EC%8A%A4%2F%EC%84%B8%EB%9F%BC%2F%EC%95%B0%ED%94%8C&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
# main_url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000200010015&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000200010015_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EB%B9%84%ED%83%80%EB%AF%BC&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
main_url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000002000100150001&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=48&searchTypeSort=btn_list&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000002000100150001_Small&amplitudePageGubun=SMALL_CATE&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EC%83%81%EC%84%B8_%EC%86%8C%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&midCategory=%EB%B9%84%ED%83%80%EB%AF%BC&smallCategory=%EC%86%8C_%EA%B8%B0%ED%83%80+%EB%B9%84%ED%83%80%EB%AF%BC&checkBrnds=&lastChkBrnd=&t_3rd_category_type=%EC%86%8C_%EA%B8%B0%ED%83%80+%EB%B9%84%ED%83%80%EB%AF%BC'
# 웹 드라이버 실행
driver = webdriver.Chrome()
# 웹 드라이버 최대화
driver.maximize_window()
# 메인 페이지로 이동
driver.get(main_url)
time.sleep(2)

## 메인 페이지 개수 구하기 ##
total_num = int(driver.find_element(By.CSS_SELECTOR, '#Contents > p > span').text)
if total_num % 48 != 0:
    main_page_num = total_num // 48 + 1
else:
    main_page_num = total_num // 48

print('총 페이지 수 :', main_page_num)

driver.quit()

총 페이지 수 : 2


## 크롤링 하기

In [ ]:
# 웹 드라이버 실행
driver = webdriver.Chrome()
# 웹 드라이버 최대화
driver.maximize_window()

for n in range(main_page_num-1):
    ## 크롤링 할 내용 저장할 리스트 ##
    # 주소 저장 리스트
    urls = []
    # 사진 저장 리스트
    pictures = []
    # 브랜드 저장 리스트
    brands = []
    # 제품명 저장 리스트
    names = []
    # 판매가 저장 리스트
    prices = []
    # 최적가 저장 리스트
    sale_prices = []
    # 용량 저장 리스트
    volumes = []
    # 주요 사양 저장 리스트
    skin_types = []
    # 성분 저장 리스트
    ingredients = []
    # 리뷰 저장 리스트
    reviews = []

    # 올리브영 - 스킨케어 - 에센스/세럼/앰플 페이지
    main_url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000002000100150001&fltDispCatNo=&prdSort=01&pageIdx={n+1}&rowsPerPage=48&searchTypeSort=btn_list&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000002000100150001_Small&amplitudePageGubun=SMALL_CATE&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EC%83%81%EC%84%B8_%EC%86%8C%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&midCategory=%EB%B9%84%ED%83%80%EB%AF%BC&smallCategory=%EC%86%8C_%EA%B8%B0%ED%83%80+%EB%B9%84%ED%83%80%EB%AF%BC&checkBrnds=&lastChkBrnd=&t_3rd_category_type=%EC%86%8C_%EA%B8%B0%ED%83%80+%EB%B9%84%ED%83%80%EB%AF%BC'

    # 메인 페이지로 이동
    driver.get(main_url)
    time.sleep(1)

    # 리스트 형태로 만드는 버튼 클릭하기
    driver.find_element(By.CSS_SELECTOR, '#Contents > div.cate_align_box > div.type_sort > button.btn_list').click()
    print(f'PAGE : [{n+1}] / [{main_page_num}]')
    # time.sleep(1)

    # 48개 있다고 설정했으니깐 for문 돌리기 (4개마다 ul 번호가 바뀐다.)
    for i in range(12):
        for j in range(4):
            product_change(i, j)

    # 데이터프레임 만들어서 저장하기
    make_save(n+1, urls, pictures, brands, names, prices, sale_prices, volumes, skin_types, ingredients, reviews)
    print('='*50)

time.sleep(1)
driver.quit()

PAGE : [1] / [2]
START >> [1] / [48]
크롤링 실패
START >> [2] / [48]
판매가 저장 실패
리뷰 저장 시작
>> [1] / [10]
>> [2] / [10]
>> [3] / [10]
>> [4] / [10]
>> [5] / [10]
>> [6] / [10]
>> [7] / [10]
>> [8] / [10]
>> [9] / [10]
>> [10] / [10]
리뷰 저장 완료
START >> [3] / [48]
판매가 저장 실패
리뷰 저장 시작
>> [1] / [10]
>> [2] / [10]
>> [3] / [10]
>> [4] / [10]
>> [5] / [10]
>> [6] / [10]
>> [7] / [10]
>> [8] / [10]
>> [9] / [10]
>> [10] / [10]
리뷰 저장 완료
START >> [4] / [48]
판매가 저장 실패
리뷰 저장 시작
>> [1] / [10]
>> [2] / [10]
>> [3] / [10]
>> [4] / [10]
>> [5] / [10]
>> [6] / [10]
>> [7] / [10]
>> [8] / [10]
>> [9] / [10]
>> [10] / [10]
리뷰 저장 완료
START >> [5] / [48]
판매가 저장 실패
크롤링 실패
START >> [6] / [48]
크롤링 실패
START >> [7] / [48]
크롤링 실패
START >> [8] / [48]
크롤링 실패
START >> [9] / [48]
판매가 저장 실패
크롤링 실패
START >> [10] / [48]
판매가 저장 실패
크롤링 실패
START >> [11] / [48]
크롤링 실패
START >> [12] / [48]
판매가 저장 실패
크롤링 실패
START >> [13] / [48]
판매가 저장 실패
크롤링 실패
START >> [14] / [48]
판매가 저장 실패
리뷰 저장 시작
>> [1] / [10]
>> [2] / [10]
>> [3] / [10]
>> [4] / [

In [ ]:
len(pictures)

43

In [ ]:
for n in range(1):
    reviews.append([])
    print(len(reviews))

10


In [ ]:
reviews

[['순해서 너무 좋으나, 유분감이 느껴져서 적당히 바르는 게 좋을 것 같아요',
  '예전에 써봤을 때 보습력이 괜찮았던거 같아서 몇년만에 재구입해봤어요ㅋㅋ 여름에 무겁지않게 바를 수 있는거 같습니다',
  '엄청 순하고 수분크림이지만 좀 되직해요. 가격도 저렵해서 좋아용',
  '무난하게 사용하기 좋아요!! 적당히 촉촉하고 좋아요!!',
  '순하고 좋아요 여름에 쓰기도 좋아요 끈적임이 없고 가격도 착해서 떨어지기전에 쟁여 두어요',
  '건성피부 정착템~~ 가격 비싼것만 빼고 다 좋아요. 할인할때 쟁여놔야함',
  '샘플 우연히 썼다가 정말 좋아서 사용하겠다고 찜해놓고 쓰고 있어요. 이 라인 다 좋아요.',
  '가성비가 좋아요 작당히 보습감도 있고 수영 다니면서 사물함에 넣아두고 사용합니다',
  '늘 쓰던 제품이에요 피부가 예민한 엄마도 순하고 좋대요 촉촉합니다',
  '로션은 요것만 사용합니다 이제 정착템으로 잘 쓰고 잇습니다..! 항상 이름헷갈려서 구입하개 되는 제품입니당 ㅋㅋㅋ 그래도 만족합니다 수부지인데 요거바르고 수분크림바르면 전혀 땡김없이 피부가 편해져서 너무 좋아요! 항상 사용합니당',
  '배송은 예상보다 늦었지만 제품은 잘 왔습니다 동생이 이제품만 사용해서 또 구매해요',
  '조아용 8년째 쓰는 제 인생템이에요 건성분들 사세요 !! 얼굴에 뭐도 안나고 수분채우기 딱',
  '대학생때부터 있었던 제품 같은데 건성피부 보습로션으로 이만한게 없는것 같아요 트러블도 안나고 자극적이지 않고 바르면 끈적임 없이 촉촉하고 부드럽게 흡수가 되요 여름에도 가볍게 바르기 좋네요',
  '조아여,,어쩌다가 사용했는데 나름 피부에 잘 맞아서 잘 사용하고 있습니다',
  '두 개째 사용중이고 하나 더 쟁여놨어요 아침 세안 후 바로 바르는 용도로 쓰는데요 촉촉하고 자극없고 흡수 잘되고.... 패키지도 심플해서 좋아요',
  '순하고 건성 피부엔 이제품 만한게 없는것 같아요.집앞에 애장에서 편하게 구입했습니다',
  '보습하면 고쿠쥰! 10년이상 고쿠쥰만 쓰고

In [ ]:
make_save(5, urls, pictures, brands, names, prices, sale_prices, volumes, skin_types, ingredients, reviews[:43])

DF 생성 완료
DF 저장 완료


In [ ]:
# //*[@id="Contents"]/ul[2]/li[1]/div/p[4]/button[2]
# //*[@id="Contents"]/ul[2]/li[2]/div/p[4]/button[2]
# //*[@id="Contents"]/ul[3]/li[1]/div/p[4]/button[2] #Contents > ul:nth-child(9) > li.flag > div > p.prd_btn_area > button.btn_new_pop.goodsList
#Contents > ul:nth-child(8) > li.flag > div > a